<a href="https://colab.research.google.com/github/solomontessema/Data-Analytics-and-AI-with-Python/blob/main/notebooks/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Install Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download Spark from a stable mirror
!wget -q https://mirrors.huaweicloud.com/apache/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Extract Spark
!tar -xvf spark-3.4.1-bin-hadoop3.tgz

# Install findspark
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AirbnbReviews").getOrCreate()


In [ ]:
df = spark.read.csv("/content/drive/MyDrive/listings.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)


In [ ]:
df.head(5)

In [ ]:
from pyspark.sql.functions import col, to_date

df_clean = df.filter(col("price").isNotNull())
df_clean = df_clean.withColumn("last_review", to_date(col("last_review")))
df_clean.select("id", "name", "price", "last_review").show(5)


In [ ]:
df_clean.groupBy("room_type").avg("price").orderBy("avg(price)", ascending=False).show()


In [ ]:
df_clean.groupBy("neighbourhood").count().orderBy("count", ascending=False).show()

In [ ]:
df_clean.write.parquet("/content/drive/MyDrive/listings_clean.parquet")



In [ ]:
df_clean.groupby("name").avg("reviews_per_month").orderBy("name",ascending =True).show()

In [ ]:
df_clean.groupBy("name").count().show()

In [ ]:
df_clean.join(df_clean,df_clean["name"]==df_clean["name"], "inner").show()
